In [ ]:
import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter
import nltk
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
os.listdir('./Database')

In [ ]:
arquivosNormais = [nomeArquivo for nomeArquivo in (os.listdir("./Database/ham")) if len(nomeArquivo) > 20]
arquivosSpam = [nomeArquivo for nomeArquivo in (os.listdir("./Database/spam")) if len(nomeArquivo) > 20]

In [ ]:
def carregarEmail(tipo, arquivo):

    caminho = ""

    if tipo == "spam":
        caminho = "./Database/spam"
    if tipo == "ham":
        caminho = "./Database/ham"

    with open(f"{caminho}/{arquivo}", "rb") as file:
        return email.parser.BytesParser(policy=email.policy.default).parse(file)

emailsNormais = [carregarEmail("ham", arquivo) for arquivo in arquivosNormais]
emailsSpam = [carregarEmail("spam", arquivo) for arquivo in arquivosSpam]

aux = emailsNormais
aux

In [ ]:
for i in aux.keys():
    print(f"{i} : {aux[i]}")

print('\n')
print('Conteudo:',aux.get_content())

In [ ]:
def get_email_structure(email):
    if isinstance(email, str):
        return email

    payload = email.get_payload()

    if isinstance(payload, list):
        for sub_email in payload:
            return "multipart({})".format(", ".join([
            get_email_structure(sub_email)]))
    else:

        return email.get_content_type()


emailsNormaisEstrutura = get_email_structure(emailsNormais[1])
emailsSpamEstrutura = get_email_structure(emailsSpam[1])

In [ ]:
emailsNormaisEstrutura

In [ ]:
emailsSpamEstrutura

In [ ]:
def html_to_text(email):
    try:
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        return soup.text.replace('\n\n','')
    except:
        return "empty"


def email_to_text(email):
    struct = get_email_structure(email)
    for part in email.walk():
        print("part", part)
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_plain(part)

In [ ]:
email_to_text()